In [133]:
import numpy as np
import pandas as pd
import sys, copy, os, shutil, time
import torch
import torch.nn as nn
import utils
from resnet import resnet20, resnet32, resnet44
from IPython.display import clear_output
from tqdm.notebook import tqdm

# for loading datasets
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, MNIST, FashionMNIST

# make a directory for logs
if "logs" not in os.listdir():
    os.mkdir("logs")
    
# NO FANCY TRICKS -- JUST RESIZE TO 32 x 32!
train_transforms = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])
    
# let's use a batch size of 60000
batch_size = 512

# USE A GPU IF POSSIBLE!
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [135]:
# for MNIST, let's see which train and test points each model got correct

# begin by loading our data
data_train = MNIST(root="./data", train=True, download=True, transform=train_transforms)
data_test = MNIST(root="./data", train=False, download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=False)
testloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False)
data_dim = 1024

# create a dataframe with N + 3 columns (model-name + variant + seed)
mnist_train_scores = pd.DataFrame(data=None, columns=["arch", "variant", "seed", "epoch"] \
                                  + list(np.arange(60000)))
mnist_test_scores = pd.DataFrame(data=None, columns=["arch", "variant", "seed", "epoch"] \
                                 + list(np.arange(10000)))

# what models do we have available for this dataset?
model_names = [f for f in sorted(os.listdir("models/MNIST")) if "seed" in f]

# go thru each of our model
for model_num, model_name in enumerate(model_names):
    
    # start time
    start = time.time()
    
    # first figure out what architecture we need to be loading
    if "cnn" in model_name:
        
        # how many modules do we have?
        variant = int(model_name.split("num-modules=")[1].split("_")[0])
        seed = int(model_name.split("seed=")[1])
        model_arch = "cnn"
        
        # load the appropriate architecture
        model = utils.MNIST_CNN(num_modules=variant)

    elif "mlp" in model_name:
        
        # how many hidden layers do we have?
        variant = int(model_name.split("num-layers=")[1].split("_")[0])
        seed = int(model_name.split("seed=")[1])
        model_arch = "mlp"
        
        # load the appropriate architecture
        model = utils.MNIST_MLP(num_layers=variant, data_dim=data_dim)
        
    elif "resnet" in model_name:
        
        # which resnet variant are we loading?
        variant = int(model_name.split("variant=")[1].split("_")[0])
        seed = int(model_name.split("seed=")[1])
        model_arch = "resnet"
        
        # load the appropriate architecture
        if variant == 20:
            model = resnet20()
        elif variant == 32:
            model = resnet32()
        elif variant == 44:
            model = resnet44()
            
    # create our row header for this row
    header = [model_arch, variant, seed]
    
    # REVISION: ONLY LOOKING AT THE LAST EPOCH:
    for epoch in range(99, 100):
        
        # load in the weights for this epoch
        model.load_state_dict(torch.load(f"models/MNIST/{model_name}/{str(epoch).zfill(3)}.pth"))
        model.to(device); model.eval()
        
        ###### TRAINING SET METRICS
        
        # create a list of one-hot encoded accuracies
        train_accs = np.array([])
        
        # compute accuracy on training set
        for data in tqdm(trainloader):
            
            # unpack our x's and y's -- concatenate if necessary
            inputs, labels = data
            if model_arch == "resnet":
                inputs = torch.cat([inputs, inputs, inputs], dim=1)
            inputs, labels = inputs.to(device), labels.to(device)
            
            # do not use grad - make our predictions + record our accuracies
            with torch.no_grad():
                outputs = model(inputs)
                _, predictions = torch.max(outputs.data, 1)
                train_accs = np.concatenate([train_accs, (predictions == labels).cpu().numpy()])
            
        # add to our row
        mnist_train_scores.loc[len(mnist_train_scores.index)] = header + [epoch] + list(train_accs)
        
        ###### TESTING SET METRICS
        
        # create a list of one-hot encoded accuracies
        test_accs = np.array([])
        
        # compute accuracy on the TEST set
        for data in tqdm(testloader):
            
            # unpack our x's and y's -- concatenate if necessary
            inputs, labels = data
            if model_arch == "resnet":
                inputs = torch.cat([inputs, inputs, inputs], dim=1)
            inputs, labels = inputs.to(device), labels.to(device)
            
            # do not use grad - make our predictions + record our accuracies
            with torch.no_grad():
                outputs = model(inputs)
                _, predictions = torch.max(outputs.data, 1)
                test_accs = np.concatenate([test_accs, (predictions == labels).cpu().numpy()])
            
        # add to our row
        mnist_test_scores.loc[len(mnist_test_scores.index)] = header + [epoch] + list(test_accs)
        
    # compute end time
    end = time.time()
        
    # status update
    clear_output(wait=True)
    print(f"Finished processing Epoch {str(epoch + 1).zfill(3)} of 100 on Model {str(model_num + 1).zfill(3)} of 450 in {np.round(end - start, 3)} seconds.")

# save our logs at the very end
mnist_train_scores.to_csv("logs/mnist_train_scores.csv", index=False)
mnist_test_scores.to_csv("logs/mnist_test_scores.csv", index=False)

Finished processing Epoch 100 of 100 on Model 450 of 450 in 24.049 seconds.
